In [1]:
!pip install -Uqq nixtla

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [5]:
# Load libraries

from nixtla import NixtlaClient
import pandas as pd
import numpy as np

In [3]:
# initiate Nixtla client

nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-fQ023pkMcbq0hsxZCUNUyHruTjNHOe07YkG6PIdlUduZaQeyXG18s1e59Fjt2FieAZBWOPxBKTgEeJeD'
)

In [4]:
# key validation

nixtla_client.validate_api_key()

True

In [8]:
# Load data

from google.colab import files
uploaded = files.upload()

Saving test_tscv.csv to test_tscv (1).csv
Saving train_tscv.csv to train_tscv (1).csv


In [21]:
# read data

train_tscv = pd.read_csv("train_tscv (1).csv")
test_tscv = pd.read_csv("test_tscv (1).csv")

# make the yearmonth column as date format

train_tscv['yearmonth'] = pd.to_datetime(train_tscv['yearmonth']) + pd.offsets.MonthEnd(0)
train_tscv = train_tscv.sort_values(by = 'yearmonth')


test_tscv['yearmonth'] = pd.to_datetime(test_tscv['yearmonth']) + pd.offsets.MonthEnd(0)
test_tscv = test_tscv.sort_values(by = 'yearmonth')


train_tscv = train_tscv.rename(columns={
    'yearmonth': 'ds',
    'stock_distributed': 'y'
})

test_tscv = test_tscv.rename(columns={
    'yearmonth': 'ds',
    'stock_distributed': 'y'
})

train_tscv.head()

<ipython-input-21-2bbf49de13b5>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_tscv['yearmonth'] = pd.to_datetime(train_tscv['yearmonth']) + pd.offsets.MonthEnd(0)
<ipython-input-21-2bbf49de13b5>:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test_tscv['yearmonth'] = pd.to_datetime(test_tscv['yearmonth']) + pd.offsets.MonthEnd(0)


,unique_id,ds,region,district,site_code,product_type,product_code,y,zero_per,site_type,year,month,lag_1,lag_2,lag_3,lag_4,lag_rolling_mean_1234,rolling_max_4,rolling_zero_per_4,horizon
0,C1014AS27137-1,2016-01-31,1,1,1,1,1,0,0.770833,1,2016,1,0,0,0,0,0.0,0,0.0,1
27090,C2062AS27000-1,2016-01-31,14,58,108,2,2,16,0.000000,2,2016,1,0,0,0,0,0.0,0,0.0,1
12255,C1072AS27134-1,2016-01-31,3,65,140,3,6,0,0.437500,1,2016,1,0,0,0,0,0.0,0,0.0,1
27047,C2062AS27138-1,2016-01-31,14,58,108,1,5,0,0.312500,2,2016,1,0,0,0,0,0.0,0,0.0,1
51561,C2068AS27138-2,2016-01-31,9,36,72,1,5,0,0.458333,2,2016,1,0,0,0,0,0.0,0,0.0,2


In [33]:
# define parameters

horizon = 3

# Define prediction intervals
levels = np.arange(0.01, 1.00, 0.01)

In [40]:
# create bounded data to tackle zero demand - this is to avoid negative forecasts

df_transformed = train_tscv.copy()

df_transformed['y'] = np.log(df_transformed['y']+1)

In [41]:
# model fit

timegpt_quantile_fcst_df = nixtla_client.forecast(
    df=df_transformed,
    h=horizon,
    quantiles=levels,
    X_df=test_tscv,
    freq = 'M',
)

/usr/local/lib/python3.10/dist-packages/nixtla/nixtla_client.py:298: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fcst_df[q_col] = fcst_df[col].values


In [45]:
# bring back to normal scale

cols_to_transform = [col for col in timegpt_quantile_fcst_df if col not in ['unique_id', 'ds']]

for col in cols_to_transform:
    timegpt_quantile_fcst_df[col] = np.exp(timegpt_quantile_fcst_df[col])-1

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)


In [46]:
timegpt_quantile_fcst_df

,unique_id,ds,TimeGPT,TimeGPT-q-1,TimeGPT-q-2,TimeGPT-q-3,TimeGPT-q-4,TimeGPT-q-5,TimeGPT-q-6,TimeGPT-q-6,...,TimeGPT-q-90,TimeGPT-q-91,TimeGPT-q-92,TimeGPT-q-93,TimeGPT-q-94,TimeGPT-q-95,TimeGPT-q-96,TimeGPT-q-97,TimeGPT-q-98,TimeGPT-q-99
0,C1014AS27137-1,2019-08-31,1.866721e+00,1.747200e+00,1.749541e+00,1.751885e+00,1.754234e+00,1.756586e+00,2.731754e+06,2.731754e+06,...,1.972076e+00,1.974805e+00,1.977540e+00,1.980279e+00,1.983023e+00,1.985772e+00,1.988525e+00,1.991284e+00,1.994047e+00,1.996816e+00
1,C1014AS27137-1,2019-09-30,2.399503e+01,2.165000e+01,2.169490e+01,2.173991e+01,2.178504e+01,2.183028e+01,inf,inf,...,2.615064e+01,2.620758e+01,2.626468e+01,2.632194e+01,2.637935e+01,2.643692e+01,2.649464e+01,2.655253e+01,2.661057e+01,2.666877e+01
2,C1014AS27137-1,2019-10-31,1.046882e+01,9.498716e+00,9.517328e+00,9.535988e+00,9.554694e+00,9.573448e+00,inf,inf,...,1.135716e+01,1.138059e+01,1.140408e+01,1.142763e+01,1.145124e+01,1.147492e+01,1.149866e+01,1.152246e+01,1.154633e+01,1.157026e+01
3,C2062AS27000-1,2019-08-31,2.392260e+12,3.647479e+08,4.233038e+08,4.918128e+08,5.720572e+08,6.661543e+08,inf,inf,...,5.313480e+16,7.107019e+16,9.526861e+16,1.279892e+17,1.723316e+17,2.325586e+17,3.145450e+17,4.264063e+17,5.793784e+17,7.890536e+17
4,C2062AS27000-1,2019-09-30,8.492659e+09,7.112866e+09,7.138550e+09,7.164332e+09,7.190211e+09,7.216187e+09,inf,inf,...,9.825304e+09,9.861290e+09,9.897413e+09,9.933674e+09,9.970074e+09,1.000661e+10,1.004329e+10,1.008011e+10,1.011707e+10,1.015417e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790,C1399AS27133-1,2019-09-30,1.440006e+00,1.436917e-01,1.497582e-01,1.560979e-01,1.627244e-01,1.696522e-01,8.669421e+00,8.669421e+00,...,6.471117e+01,7.649249e+01,9.098184e+01,1.089203e+02,1.312825e+02,1.593599e+02,1.948769e+02,2.401538e+02,2.983393e+02,3.737416e+02
6791,C1399AS27133-1,2019-10-31,1.762280e+00,5.178604e-02,5.514648e-02,5.873101e-02,6.255536e-02,6.663648e-02,5.902468e+00,5.902468e+00,...,1.307182e+05,2.751589e+05,6.070892e+05,1.408101e+06,3.444284e+06,8.914707e+06,2.450240e+07,7.178804e+07,2.251077e+08,7.587294e+08
6792,C1077AS27134-2,2019-09-30,2.257807e+08,5.978746e+05,6.610925e+05,7.315503e+05,8.101371e+05,8.978582e+05,inf,inf,...,1.945637e+11,2.367935e+11,2.886173e+11,3.523096e+11,4.307061e+11,5.273476e+11,6.466619e+11,7.941945e+11,9.769019e+11,1.203523e+12
6793,C1077AS27134-2,2019-10-31,2.072103e+06,8.062729e+05,8.214402e+05,8.369142e+05,8.527016e+05,8.688089e+05,inf,inf,...,4.699496e+06,4.799481e+06,4.901734e+06,5.006311e+06,5.113267e+06,5.222659e+06,5.334546e+06,5.448988e+06,5.566048e+06,5.685788e+06


In [47]:
# save file

timegpt_quantile_fcst_df.to_csv('timegpt_quantile_cv.csv', index = False)
files.download("timegpt_quantile_cv.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>